In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd "/content/drive/My Drive/Data Job/Colab Notebooks/Data Science Nanodegree/Exercise Files/udacity-nanodegree-exercise/5_Experimental_Design_and_Recommendations/Lesson_6_Intro_to_Recommendation/"


/content/drive/My Drive/Data Job/Colab Notebooks/Data Science Nanodegree/Exercise Files/udacity-nanodegree-exercise/5_Experimental_Design_and_Recommendations/Lesson_6_Intro_to_Recommendation


In [4]:
%ls

 all_recs.p                                       movies_clean.csv
'Collaborative Filtering.ipynb'                   movies.dat
 helper.py                                        __pycache__/
'Introduction to the Recommendation Data.ipynb'   ratings.dat
'Measuring Similarity.ipynb'                      reviews_clean.csv
'Measuring Similarity - Solution.ipynb'           tests.py
 Most_Popular_Recommendations.ipynb


## Recommendations with MovieTweetings: Collaborative Filtering

One of the most popular methods for making recommendations is **collaborative filtering**.  In collaborative filtering, you are using the collaboration of user-item recommendations to assist in making new recommendations.  

There are two main methods of performing collaborative filtering:

1. **Neighborhood-Based Collaborative Filtering**, which is based on the idea that we can either correlate items that are similar to provide recommendations or we can correlate users to one another to provide recommendations.

2. **Model Based Collaborative Filtering**, which is based on the idea that we can use machine learning and other mathematical models to understand the relationships that exist amongst items and users to predict ratings and provide ratings.


In this notebook, you will be working on performing **neighborhood-based collaborative filtering**.  There are two main methods for performing collaborative filtering:

1. **User-based collaborative filtering:** In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

2. **Item-based collaborative filtering:** In this type of recommendation, first you need to find the items that are most related to each other item (based on similar ratings).  Then you can use the ratings of an individual on those similar items to understand if a user will like the new item.

In this notebook you will be implementing **user-based collaborative filtering**.  However, it is easy to extend this approach to make recommendations using **item-based collaborative filtering**.  First, let's read in our data and necessary libraries.

**NOTE**: Because of the size of the datasets, some of your code cells here will take a while to execute, so be patient!

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from scipy.sparse import csr_matrix
from IPython.display import HTML


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

print(reviews.head())

   user_id  movie_id  rating   timestamp                 date  month_1  \
0        1     68646      10  1381620027  2013-10-12 23:20:27        0   
1        1    113277      10  1379466669  2013-09-18 01:11:09        0   
2        2    422720       8  1412178746  2014-10-01 15:52:26        0   
3        2    454876       8  1394818630  2014-03-14 17:37:10        0   
4        2    790636       7  1389963947  2014-01-17 13:05:47        0   

   month_2  month_3  month_4  month_5  ...  month_9  month_10  month_11  \
0        0        0        0        0  ...        0         1         0   
1        0        0        0        0  ...        0         0         0   
2        0        0        0        0  ...        0         1         0   
3        0        0        0        0  ...        0         0         0   
4        0        0        0        0  ...        0         0         0   

   month_12  year_2013  year_2014  year_2015  year_2016  year_2017  year_2018  
0         0          1  

### Measures of Similarity

When using **neighborhood** based collaborative filtering, it is important to understand how to measure the similarity of users or items to one another.  

There are a number of ways in which we might measure the similarity between two vectors (which might be two users or two items).  In this notebook, we will look specifically at two measures used to compare vectors:

* **Pearson's correlation coefficient**

Pearson's correlation coefficient is a measure of the strength and direction of a linear relationship. The value for this coefficient is a value between -1 and 1 where -1 indicates a strong, negative linear relationship and 1 indicates a strong, positive linear relationship. 

If we have two vectors x and y, we can define the correlation between the vectors as:


$$CORR(x, y) = \frac{\text{COV}(x, y)}{\text{STDEV}(x)\text{ }\text{STDEV}(y)}$$

where 

$$\text{STDEV}(x) = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}$$

and 

$$\text{COV}(x, y) = \frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

where n is the length of the vector, which must be the same for both x and y and $\bar{x}$ is the mean of the observations in the vector.  

We can use the correlation coefficient to indicate how alike two vectors are to one another, where the closer to 1 the coefficient, the more alike the vectors are to one another.  There are some potential downsides to using this metric as a measure of similarity.  You will see some of these throughout this workbook.


* **Euclidean distance**

Euclidean distance is a measure of the straightline distance from one vector to another.  Because this is a measure of distance, larger values are an indication that two vectors are different from one another (which is different than Pearson's correlation coefficient).

Specifically, the euclidean distance between two vectors x and y is measured as:

$$ \text{EUCL}(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

Different from the correlation coefficient, no scaling is performed in the denominator.  Therefore, you need to make sure all of your data are on the same scale when using this metric.

**Note:** Because measuring similarity is often based on looking at the distance between vectors, it is important in these cases to scale your data or to have all data be in the same scale.  If some measures are on a 5 point scale, while others are on a 100 point scale, you are likely to have non-optimal results due to the difference in variability of your features.  In this case, we will not need to scale data because they are all on a 10 point scale, but it is always something to keep in mind!

------------

### User-Item Matrix

In order to calculate the similarities, it is common to put values in a matrix.  In this matrix, users are identified by each row, and items are represented by columns.  


![alt text](images/userxitem.png "User Item Matrix")


In the above matrix, you can see that **User 1** and **User 2** both used **Item 1**, and **User 2**, **User 3**, and **User 4** all used **Item 2**.  However, there are also a large number of missing values in the matrix for users who haven't used a particular item.  A matrix with many missing values (like the one above) is considered **sparse**.

Our first goal for this notebook is to create the above matrix with the **reviews** dataset.  However, instead of 1 values in each cell, you should have the actual rating.  

The users will indicate the rows, and the movies will exist across the columns. To create the user-item matrix, we only need the first three columns of the **reviews** dataframe, which you can see by running the cell below.

In [3]:
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_items.head()

,user_id,movie_id,rating
0,1,68646,10
1,1,113277,10
2,2,422720,8
3,2,454876,8
4,2,790636,7


### Creating the User-Item Matrix

In order to create the user-items matrix (like the one above), I personally started by using a [pivot table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html). 

However, I quickly ran into a memory error (a common theme throughout this notebook).  I will help you navigate around many of the errors I had, and achieve useful collaborative filtering results! 

_____

`1.` Create a matrix where the users are the rows, the movies are the columns, and the ratings exist in each cell, or a NaN exists in cells where a user hasn't rated a particular movie. If you get a memory error (like I did), [this link here](https://stackoverflow.com/questions/39648991/pandas-dataframe-pivot-memory-error) might help you!

In [4]:
# Create user-by-item matrix
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

Check your results below to make sure your matrix is ready for the upcoming sections.

In [5]:
assert movies.shape[0] == user_by_movie.shape[1], "Oh no! Your matrix should have {} columns, and yours has {}!".format(movies.shape[0], user_by_movie.shape[1])
assert reviews.user_id.nunique() == user_by_movie.shape[0], "Oh no! Your matrix should have {} rows, and yours has {}!".format(reviews.user_id.nunique(), user_by_movie.shape[0])
print("Looks like you are all set! Proceed!")
HTML('<img src="images/greatjob.webp">')

Looks like you are all set! Proceed!


`2.` Now that you have a matrix of users by movies, use this matrix to create a dictionary where the key is each user and the value is an array of the movies each user has rated.

In [6]:
user_by_movie.head(5)

movie_id,8,10,12,25,91,417,439,443,628,833,...,8144778,8144868,8206708,8289196,8324578,8335880,8342748,8342946,8402090,8439854
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
user_id = 3
user_by_movie.loc[user_id][user_by_movie.loc[user_id].notnull()]

movie_id
1790864    7.0
2170439    7.0
2203939    6.0
Name: 3, dtype: float64

In [14]:
# Create a dictionary with users and corresponding movies seen

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    
    return user_by_movie.loc[user_id][user_by_movie.loc[user_id].notnull()]


def create_user_movie_dict():
    '''
    INPUT: None
    OUTPUT: movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    Creates the movies_seen dictionary
    '''
    movies_seen = {}
    
    for user in user_by_movie.index:
        movies_seen[user] = movies_watched(user)
    
    return movies_seen


# Use your function to return dictionary
movies_seen = create_user_movie_dict()

In [72]:
movies_seen[45].shape

(15,)

`3.` If a user hasn't rated more than 2 movies, we consider these users "too new".  Create a new dictionary that only contains users who have rated more than 2 movies.  This dictionary will be used for all the final steps of this workbook.

In [26]:
# Remove individuals who have watched 2 or fewer movies - don't have enough data to make recs

def create_movies_to_analyze(movies_seen, lower_bound=2):
    '''
    INPUT:  
    movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    lower_bound - (an int) a user must have more movies seen than the lower bound to be added to the movies_to_analyze dictionary

    OUTPUT: 
    movies_to_analyze - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    The movies_seen and movies_to_analyze dictionaries should be the same except that the output dictionary has removed 
    
    '''
    
    movies_to_analyze = { key:value for (key,value) in movies_seen.items() if value.shape[0] > lower_bound }
    
    return movies_to_analyze


# Use your function to return your updated dictionary
movies_to_analyze = create_movies_to_analyze(movies_seen)

In [74]:
# Run the tests below to check that your movies_to_analyze matches the solution
assert len(movies_to_analyze) == 23512, "Oops!  It doesn't look like your dictionary has the right number of individuals."
assert len(movies_to_analyze[2]) == 23, "Oops!  User 2 didn't match the number of movies we thought they would have."
assert len(movies_to_analyze[7])  == 3, "Oops!  User 7 didn't match the number of movies we thought they would have."
print("If this is all you see, you are good to go!")

If this is all you see, you are good to go!


### Calculating User Similarities

Now that you have set up the **movies_to_analyze** dictionary, it is time to take a closer look at the similarities between users.  Below is the pseudocode for how I thought about determining the similarity between users:

```
for user1 in movies_to_analyze
    for user2 in movies_to_analyze
        see how many movies match between the two users
        if more than two movies in common
            pull the overlapping movies
            compute the distance/similarity metric between ratings on the same movies for the two users
            store the users and the distance metric
```

However, this took a very long time to run, and other methods of performing these operations did not fit on the workspace memory!

Therefore, rather than creating a dataframe with all possible pairings of users in our data, your task for this question is to look at a few specific examples of the correlation between ratings given by two users.  For this question consider you want to compute the [correlation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) between users.

`4.` Using the **movies_to_analyze** dictionary and **user_by_movie** dataframe, create a function that computes the correlation between the ratings of similar movies for two users.  Then use your function to compare your results to ours using the tests below.  

In [136]:
from scipy.stats import spearmanr, kendalltau
user1 = 2
user2 = 2
common_movies = pd.concat([movies_to_analyze[user1], movies_to_analyze[user2]], axis=1, join="inner")
common_movies.corr()[user1][user2]
all(movies_to_analyze[user1] == movies_to_analyze[user2])

True

In [143]:
np.intersect1d(movies_to_analyze[user1], movies_to_analyze[user2], assume_unique=True)

array([ 5.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  8.,
        8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,
        8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  9., 10.,
       10., 10.])

In [139]:
def compute_correlation(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    common_movies = pd.concat([movies_to_analyze[user1], movies_to_analyze[user2]], axis=1, join="inner")
    if user1 == user2:
        return 1 
    return common_movies.corr()[user1][user2] #return the correlation

In [598]:
user_by_movie.loc[[user1, user2], [10]]
# sim_mov

movie_id,10
user_id,
2,NaN
104,NaN


In [601]:
movies_to_analyze[user1].index

Int64Index([ 422720,  454876,  790636,  816711, 1091191, 1103275, 1322269,
            1390411, 1398426, 1431045, 1433811, 1454468, 1535109, 1675434,
            1798709, 2017038, 2024544, 2294629, 2361509, 2381249, 2726560,
            2883512, 3079380],
           dtype='int64', name='movie_id')

In [614]:
user1 = 2
user2 = 66
movies1 = movies_to_analyze[user1].index
movies2 = movies_to_analyze[user2].index

sim_mov = np.intersect1d(movies1, movies2, assume_unique = True)

user_by_movie.loc[[user1, user2], sim_mov]

# df.transpose().corr().iloc[0,1]

movie_id,1454468,1798709,2883512
user_id,,,
2,8.0,10.0,8.0
66,6.0,9.0,8.0


In [619]:
np.sum((movies_to_analyze[user1] - movies_to_analyze[user2]) **2)

5.0

In [140]:
# Test your function against the solution
assert compute_correlation(2,2) == 1.0, "Oops!  The correlation between a user and itself should be 1.0."
assert round(compute_correlation(2,66), 2) == 0.76, "Oops!  The correlation between user 2 and 66 should be about 0.76."
assert np.isnan(compute_correlation(2,104)), "Oops!  The correlation between user 2 and 104 should be a NaN."

print("If this is all you see, then it looks like your function passed all of our tests!")

If this is all you see, then it looks like your function passed all of our tests!


### Why the NaN's?

If the function you wrote passed all of the tests, then you have correctly set up your function to calculate the correlation between any two users.  

`5.` But one question is, why are we still obtaining **NaN** values?  As you can see in the code cell above, users 2 and 104 have a correlation of **NaN**. Why?

Think and write your ideas here about why these NaNs exist, and use the cells below to do some coding to validate your thoughts. You can check other pairs of users and see that there are actually many NaNs in our data - 2,526,710 of them in fact. These NaN's ultimately make the correlation coefficient a less than optimal measure of similarity between two users.




In [454]:
# Which movies did both user 2 and user 104 see?
user1 = 2
user2 = 104
mov_2_104 = pd.concat([movies_to_analyze[user1], movies_to_analyze[user2]], axis=1, join="inner").reset_index()

In [460]:
# What were the ratings for each user for those movies?
mov_2_104['movie_id']
movies[movies['movie_id'].isin(mov_2_104['movie_id'])]['movie']

13676          Life of Pi (2012)
14754         World War Z (2013)
17747             Gravity (2013)
18168    Captain Phillips (2013)
Name: movie, dtype: object

`6.` Because the correlation coefficient proved to be less than optimal for relating user ratings to one another, we could instead calculate the euclidean distance between the ratings.  I found [this post](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) particularly helpful when I was setting up my function.  This function should be very similar to your previous function.  When you feel confident with your function, test it against our results.

In [167]:
def compute_euclidean_dist(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the euclidean distance between user1 and user2
    '''
    return np.sqrt(np.sum((movies_to_analyze[user1] - movies_to_analyze[user2]) ** 2))
    

In [625]:
def compute_euclidean_dist(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the euclidean distance between user1 and user2
    '''
    movies1 = movies_to_analyze[user1].index
    movies2 = movies_to_analyze[user2].index

    sim_mov = np.intersect1d(movies1, movies2, assume_unique = True)

    df = user_by_movie.loc[[user1, user2], sim_mov]

    
    return np.linalg.norm(df.loc[user1] - df.loc[user2])
    

In [626]:
compute_euclidean_dist(2,2)

0.0

In [19]:
# Read in solution euclidean distances
import pickle
df_dists = pd.read_pickle("dists.p")

In [20]:
# Test your function against the solution
assert compute_euclidean_dist(2,2) == df_dists.query("user1 == 2 and user2 == 2")['eucl_dist'][0], "Oops!  The distance between a user and itself should be 0.0."
assert round(compute_euclidean_dist(2,66), 2) == round(df_dists.query("user1 == 2 and user2 == 66")['eucl_dist'][1], 2), "Oops!  The distance between user 2 and 66 should be about 2.24."
assert np.isnan(compute_euclidean_dist(2,104)) == np.isnan(df_dists.query("user1 == 2 and user2 == 104")['eucl_dist'][4]), "Oops!  The distance between user 2 and 104 should be 2."

print("If this is all you see, then it looks like your function passed all of our tests!")

NameError: name 'compute_euclidean_dist' is not defined

### Using the Nearest Neighbors to Make Recommendations

In the previous question, you read in **df_dists**. Therefore, you have a measure of distance between each user and every other user. This dataframe holds every possible pairing of users, as well as the corresponding euclidean distance.

Because of the **NaN** values that exist within the correlations of the matching ratings for many pairs of users, as we discussed above, we will proceed using **df_dists**. You will want to find the users that are 'nearest' each user.  Then you will want to find the movies the closest neighbors have liked to recommend to each user.

I made use of the following objects:

* df_dists (to obtain the neighbors)
* user_items (to obtain the movies the neighbors and users have rated)
* movies (to obtain the names of the movies)

`7.` Complete the functions below, which allow you to find the recommendations for any user.  There are five functions which you will need:

* **find_closest_neighbors** - this returns a list of user_ids from closest neighbor to farthest neighbor using euclidean distance


* **movies_liked** - returns an array of movie_ids


* **movie_names** - takes the output of movies_liked and returns a list of movie names associated with the movie_ids


* **make_recommendations** - takes a user id and goes through closest neighbors to return a list of movie names as recommendations


* **all_recommendations** = loops through every user and returns a dictionary of with the key as a user_id and the value as a list of movie recommendations

In [273]:
user = 2
df_dists[df_dists["user1"] == user].sort_values('eucl_dist', ascending = False)

,user1,user2,eucl_dist
2743,2,52737,11.916375
1699,2,32494,11.532563
1915,2,36807,10.862780
165,2,3514,10.440307
2575,2,49739,10.344080
...,...,...,...
1590,2,30884,0.000000
1729,2,33207,0.000000
1719,2,32951,0.000000
35,2,755,0.000000


In [209]:
user_id = 2
min_rating = 7
movies_to_analyze[user_id][movies_to_analyze[user_id] >= min_rating].index[0]

422720

In [191]:
movie_ids = [10, 12]
movies[movies['movie_id'].isin(movie_ids)]['movie']

1    La sortie des usines Lumière (1895)
2          The Arrival of a Train (1896)
Name: movie, dtype: object

In [10]:
user_id = 2
min_rating=7

user_items.query("user_id == @user_id and rating >= @min_rating")['movie_id']

2      422720
3      454876
4      790636
5      816711
6     1091191
7     1103275
8     1322269
9     1390411
10    1398426
11    1431045
12    1433811
13    1454468
14    1535109
15    1675434
16    1798709
17    2017038
18    2024544
19    2294629
20    2361509
22    2726560
23    2883512
24    3079380
Name: movie_id, dtype: int64

In [65]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # I treated ties as arbitrary and just kept whichever was easiest to keep using the head method
    # You might choose to do something less hand wavy - order the neighbors
    
    
    
    return np.array(df_dists[df_dists["user1"] == user].sort_values('eucl_dist')['user2'].to_list())[1:]
    
    
    
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
    movies_liked - an array of movies the user has watched and liked
    '''
  
    return movies_to_analyze[user_id][movies_to_analyze[user_id] >= min_rating].index


def movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''

    return movies[movies['movie_id'].isin(movie_ids)]['movie']
    
    
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    movies_seen = movies_watched(user).index
    closest_neighbors = find_closest_neighbors(user)
    recs = np.array([])
    
    for neighbor in closest_neighbors:
        neigh_likes = movies_liked(neighbor)
        new_recs = np.setdiff1d(neigh_likes, movies_seen, assume_unique = True)
        
        recs = np.unique(np.concatenate([new_recs, recs], axis = 0))
        
        if len(recs) > num_recs - 1:
            break
    return movie_names(recs).to_list()


def all_recommendations(list_of_users, num_recs=10):
    '''
    INPUT 
        num_recs (int) the (max) number of recommendations for each user
    OUTPUT
        all_recs - a dictionary where each key is a user_id and the value is an array of recommended movie titles
    '''
    
    
    all_recs = {}
    
    for user in movies_seen:
        all_recs[user] = make_recommendations(user, num_recs=10)
    
    
    return all_recs

# all_recs = all_recommendations(10)

In [66]:
user = 2
make_recommendations(user)
# movies_seen = movies_watched(user).index
# closest_neighbors = find_closest_neighbors(user)
# neigh_likes = movies_liked(closest_neighbors[0])

# movie_names(np.setdiff1d(neigh_likes, movies_seen))

# neigh_likes

['Philadelphia (1993)',
 'Training Day (2001)',
 'About Schmidt (2002)',
 'Insomnia (2002)',
 'The United States of Leland (2003)',
 'Shattered Glass (2003)',
 'Man on Fire (2004)',
 'Flipped (2010)',
 'Silver Linings Playbook (2012)',
 'Lawless (2012)',
 '50/50 (2011)',
 'Crazy, Stupid, Love. (2011)',
 'The Perks of Being a Wallflower (2012)',
 'Before I Go to Sleep (2014)',
 'Zero Dark Thirty (2012)',
 'American Hustle (2013)',
 'Django Unchained (2012)',
 'Side Effects (2013)',
 'Gone Girl (2014)',
 'Enough Said (2013)',
 'Nightcrawler (2014)']

In [445]:
user = 1
find_closest_neighbors(user)

array([], dtype=float64)

In [35]:
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    recommendations = []
    
  
    #we got the closest match
    if len(find_closest_neighbors(user)) < 2:
        return recommendations
    neighbor_liked = movie_names(movies_liked(find_closest_neighbors(user)[1]))
    movies_user_seen = movie_names(pd.DataFrame(movies_seen[user]).reset_index()['movie_id'])
    recs = neighbor_liked[~neighbor_liked.isin(movies_user_seen)].to_list()
    if len(recs) >= num_recs:
        return recs[0:num_recs]
    else:
        return recs
    

In [395]:
user = 2

755

In [409]:
neighbor_liked = movie_names(movies_liked(find_closest_neighbors(user)[1]))

In [408]:
movies_user_seen = movie_names(pd.DataFrame(movies_seen[user]).reset_index()['movie_id'])

In [433]:
recs = neighbor_liked[~neighbor_liked.isin(movies_user_seen)]

In [64]:
user = 2
num_recs = 50
list_of_mov = make_recommendations(user)
print(list_of_mov)

7725                        Philadelphia (1993)
9419                        Training Day (2001)
10897                      About Schmidt (2002)
11170                           Insomnia (2002)
11507        The United States of Leland (2003)
11783                    Shattered Glass (2003)
11850                        Man on Fire (2004)
14755                            Flipped (2010)
15683            Silver Linings Playbook (2012)
16502                            Lawless (2012)
17011                              50/50 (2011)
18368               Crazy, Stupid, Love. (2011)
18892    The Perks of Being a Wallflower (2012)
19342               Before I Go to Sleep (2014)
19743                   Zero Dark Thirty (2012)
19785                    American Hustle (2013)
20115                   Django Unchained (2012)
20970                       Side Effects (2013)
22175                          Gone Girl (2014)
22921                        Enough Said (2013)
24385                       Nightcrawler

In [32]:
# This loads our solution dictionary so you can compare results
all_recs_sol = pd.read_pickle("all_recs.p")

In [63]:
all_recs_sol[2]

['Philadelphia (1993)',
 'Training Day (2001)',
 'About Schmidt (2002)',
 'Insomnia (2002)',
 'The United States of Leland (2003)',
 'Shattered Glass (2003)',
 'Man on Fire (2004)',
 'Flipped (2010)',
 'Silver Linings Playbook (2012)',
 'Lawless (2012)',
 '50/50 (2011)',
 'Crazy, Stupid, Love. (2011)',
 'The Perks of Being a Wallflower (2012)',
 'Before I Go to Sleep (2014)',
 'Zero Dark Thirty (2012)',
 'American Hustle (2013)',
 'Django Unchained (2012)',
 'Side Effects (2013)',
 'Gone Girl (2014)',
 'Enough Said (2013)',
 'Nightcrawler (2014)']

In [68]:
assert all_recs_sol[2] == make_recommendations(2), "Oops!  Your recommendations for user 2 didn't match ours."
assert all_recs_sol[26] == make_recommendations(26), "Oops!  It actually wasn't possible to make any recommendations for user 26."
assert all_recs_sol[1503] == make_recommendations(1503), "Oops! Looks like your solution for user 1503 didn't match ours."
print("If you made it here, you now have recommendations for many users using collaborative filtering!")
HTML('<img src="images/greatjob.webp">')

If you made it here, you now have recommendations for many users using collaborative filtering!


### Now What?

If you made it this far, you have successfully implemented a solution to making recommendations using collaborative filtering. 

`8.` Let's do a quick recap of the steps taken to obtain recommendations using collaborative filtering.  

In [431]:
# Check your understanding of the results by correctly filling in the dictionary below
a = "pearson's correlation and spearman's correlation"
b = 'item based collaborative filtering'
c = "there were too many ratings to get a stable metric"
d = 'user based collaborative filtering'
e = "euclidean distance and pearson's correlation coefficient"
f = "manhattan distance and euclidean distance"
g = "spearman's correlation and euclidean distance"
h = "the spread in some ratings was zero"
i = 'content based recommendation'

sol_dict = {
    'The type of recommendation system implemented here was a ...': d,# letter here,
    'The two methods used to estimate user similarity were: ': e,# letter here,
    'There was an issue with using the correlation coefficient.  What was it?': h# letter here
}

t.test_recs(sol_dict)

"That's right! All of your solutions look good!"

Additionally, let's take a closer look at some of the results.  There are two solution files that you read in to check your results, and you created these objects

* **df_dists** - a dataframe of user1, user2, euclidean distance between the two users
* **all_recs_sol** - a dictionary of all recommendations (key = user, value = list of recommendations)  

`9.` Use these two objects along with the cells below to correctly fill in the dictionary below and complete this notebook!

In [525]:
a = 567
b = 1503
c = 1319
d = 1325
e = 2526710
f = 0
g = 'Use another method to make recommendations - content based, knowledge based, or model based collaborative filtering'

sol_dict2 = {
    'For how many pairs of users were we not able to obtain a measure of similarity using correlation?': e, # letter here,
    'For how many pairs of users were we not able to obtain a measure of similarity using euclidean distance?': f, # letter here,
    'For how many users were we unable to make any recommendations for using collaborative filtering?': c,# letter here,
    'For how many users were we unable to make 10 recommendations for using collaborative filtering?': d,# letter here,
    'What might be a way for us to get 10 recommendations for every user?': g# letter here   
}

t.test_recs2(sol_dict2)

"That's right! All of your solutions look good!"

In [514]:
len(all_recs_sol.keys())

23512

In [518]:
newDict = { key:value for (key,value) in all_recs_sol.items() if len(value) < 10}

In [519]:
len(newDict.keys())

1325

In [507]:
all_possible_pairs - df_dists.shape[0]

2880677574

In [500]:
pd.isnull(compute_correlation(user1, user2))

True

In [461]:
# Use the cells below for any work you need to do!

compute_correlation(2,104)

nan

In [ ]:
# Users without recs
compute_correlation(2,104)

In [ ]:
# NaN euclidean distance values


In [ ]:
# Users with less than 10 recs
